In [1]:
import json
import pandas

## Load data

In [48]:
# Incoming transfer packages

# raw_incoming_transfer_packages_dataframe_1 = pandas.read_excel('data/royal/20210926/royal_apothecary_incoming_transfer_packages_20200101_20201231.xlsx', header=0)
# raw_incoming_transfer_packages_dataframe_2 = pandas.read_excel('data/royal/20210926/royal_apothecary_incoming_transfer_packages_20210101_20210926.xlsx', header=0)
# raw_incoming_transfer_packages_dataframe = pandas.concat([
#     raw_incoming_transfer_packages_dataframe_1,
#     raw_incoming_transfer_packages_dataframe_2,
# ])

raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/royal/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
print(f'Loaded in {len(raw_incoming_transfer_packages_dataframe.index)} incoming transfer packages...')

/Users/warren/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Loaded in 5484 incoming transfer packages...


In [51]:
# Sales transactions

raw_sales_transactions_dataframe_1 = pandas.read_excel('data/royal/20210930/royal_apothecary_sales_transactions_20200101_20201231.xlsx', header=0)
raw_sales_transactions_dataframe_2 = pandas.read_excel('data/royal/20210930/royal_apothecary_sales_transactions_20210101_20210930.xlsx', header=0)
raw_sales_transactions_dataframe = pandas.concat([
    raw_sales_transactions_dataframe_1,
    raw_sales_transactions_dataframe_2,
])
print(f'Loaded in {len(raw_sales_transactions_dataframe.index)} sales transactions...')

Loaded in 55187 sales transactions...


In [52]:
# Active inventory

raw_active_inventory_packages_dataframe = pandas.read_excel('data/royal/20210930/royal_apothecary_active_inventory_packages_20210930.xlsx', header=0)
print(f'Loaded in {len(raw_active_inventory_packages_dataframe.index)} active inventory packages...')

Loaded in 3364 active inventory packages...


## Format data

In [53]:
incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]

sales_transactions_dataframe = raw_sales_transactions_dataframe[[
#     'date_type',
#     'id',
    'receipt_id',
    'receipt_number',
#     'receipt_type',
    'sales_customer_type',
    'sales_datetime',
    'total_packages',
    'total_price',
    'tx_type',
    'tx_package_id',
    'tx_package_label',
    'tx_product_name',
    'tx_product_category_name',
    'tx_unit_of_measure',
    'tx_quantity_sold',
    'tx_total_price',
]]
sales_transactions_dataframe = sales_transactions_dataframe.rename(columns={'tx_package_id': 'package_id'})

active_inventory_packages_dataframe = raw_active_inventory_packages_dataframe[[
    'identifier',
    'license_number',
    'last_modified_at',
    'package_id',
    'package_label',
    'type',
    'packaged_date',
    'package_type',
    'product_name',
    'product_category_name',
    'quantity',
    'unit_of_measure'
]]

In [54]:
fn = lambda row: f'{row.shipper_facility_name} ({row.shipper_facility_license_number})'
col = incoming_transfer_packages_dataframe.apply(fn, axis=1)
incoming_transfer_packages_dataframe = incoming_transfer_packages_dataframe.assign(shipper_facility=col.values)

fn = lambda row: f'{row.created_date.year}-{"0" if row.created_date.month < 10 else ""}{row.created_date.month}'
col = incoming_transfer_packages_dataframe.apply(fn, axis=1)
incoming_transfer_packages_dataframe = incoming_transfer_packages_dataframe.assign(created_month=col.values)

fn = lambda row: f'{row.sales_datetime.year}-{"0" if row.sales_datetime.month < 10 else ""}{row.sales_datetime.month}'
col = sales_transactions_dataframe.apply(fn, axis=1)
sales_transactions_dataframe = sales_transactions_dataframe.assign(sales_month=col.values)

fn = lambda row: f'{row.sales_datetime.year}-{"0" if row.sales_datetime.month < 10 else ""}{row.sales_datetime.month}-{"0" if row.sales_datetime.day < 10 else ""}{row.sales_datetime.day}'
col = sales_transactions_dataframe.apply(fn, axis=1)
sales_transactions_dataframe = sales_transactions_dataframe.assign(sales_date=col.values)

## Filter data

In [55]:
# Note: we filter OUT incoming transfers if any of the following are true:
# 1. Package shipment package state is NOT Accepted.
# 2. Product name contains "Trade" or "Sample" in it. NOT IMPLEMENTED YET
# 3. Package wholesale shipper price is $0.01 AND shipped quantity is 1.
incoming_transfer_packages_dataframe = incoming_transfer_packages_dataframe[
    incoming_transfer_packages_dataframe['shipment_package_state'] == 'Accepted'
][
    (incoming_transfer_packages_dataframe['shipper_wholesale_price'] != 0.01) | (incoming_transfer_packages_dataframe['shipped_quantity'] != 1)
]
print(f'# filtered incoming transfer packages: {len(incoming_transfer_packages_dataframe)}')

# filtered incoming transfer packages: 4466


/Users/warren/venvs/bespoke/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [56]:
# Note: we filter OUT sales transactions if any of the following are true:
# 1. Sales price is $0.01.
sales_transactions_dataframe = sales_transactions_dataframe[
    sales_transactions_dataframe['tx_total_price'] != 0.01
]
print(f'# filtered sales transactions: {len(sales_transactions_dataframe)}')

# filtered sales transactions: 55174


In [57]:
# TODO: filter active inventory?

## Join data

In [58]:
sales_transactions_with_incoming_transfer_packages_dataframe = sales_transactions_dataframe.join(incoming_transfer_packages_dataframe.set_index('package_id'), on='package_id', how='inner', lsuffix='_l', rsuffix='_r')
print(f'# sales transactions with associated incoming transfer package: {len(sales_transactions_with_incoming_transfer_packages_dataframe.index)} ({len(sales_transactions_with_incoming_transfer_packages_dataframe.index) / len(sales_transactions_dataframe.index) * 100}%)')


# sales transactions with associated incoming transfer package: 5993 (10.862000217493748%)


In [59]:
sales_transactions_with_incoming_transfer_packages_dataframe = sales_transactions_with_incoming_transfer_packages_dataframe[[
#     'manifest_number',
    'package_id',
    'package_label',
    'created_month',
    'created_date',
    'shipper_facility',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'shipped_quantity',
    'sales_month',
    'sales_date',
    'sales_datetime',
    'tx_product_name',
    'tx_product_category_name',
    'tx_unit_of_measure',
    'tx_quantity_sold',
    'tx_total_price',
]]

In [60]:
active_inventory_packages_with_incoming_transfer_packages_dataframe = active_inventory_packages_dataframe.join(incoming_transfer_packages_dataframe.set_index('package_id'), on='package_id', how='inner', lsuffix='_l', rsuffix='_r')
print(f'# active inventory packages with associated transfer package: {len(active_inventory_packages_with_incoming_transfer_packages_dataframe.index)} ({len(active_inventory_packages_with_incoming_transfer_packages_dataframe.index) / len(active_inventory_packages_dataframe.index) * 100}%)')


# active inventory packages with associated transfer package: 2600 (77.28894173602853%)


## Analyze data

1. What are the top selling products with over 3 purchased packages? In 2020 vs 2021?
2. What percent of active inventory packages can we compute runout dates for? For those we can, what are the expected runout dates of the active inventory packages?

In [61]:
# 1. What percent of inventory packages do we have an incoming transfer for (ideally this is 100%)?

unique_incoming_transfer_package_package_ids = set(incoming_transfer_packages_dataframe.package_id.unique())
print(f'# unique incoming transfer package package ID: {len(unique_incoming_transfer_package_package_ids)}')

unique_incoming_transfer_package_product_names = set(incoming_transfer_packages_dataframe.product_name.unique())
print(f'# unique incoming transfer package product name: {len(unique_incoming_transfer_package_product_names)}')

unique_inventory_package_package_ids = set(active_inventory_packages_dataframe.package_id.unique())
print(f'# unique active inventory package package ID: {len(unique_inventory_package_package_ids)}')

active_inventory_package_records = active_inventory_packages_dataframe.to_dict('records')

match_count = 0
for active_inventory_package in active_inventory_package_records:
    package_id = active_inventory_package['package_id']
    if package_id in unique_incoming_transfer_package_package_ids:
        match_count += 1

print(f'# active inventory package with matching incoming transfer package: {match_count} ({match_count / len(active_inventory_package_records) * 100}%)')

# unique incoming transfer package package ID: 4466
# unique incoming transfer package product name: 2730
# unique active inventory package package ID: 3364
# active inventory package with matching incoming transfer package: 2600 (77.28894173602853%)


In [62]:
# 2. What percent of packages do we have a sales transaction for (% packages in inventory with some sales)?

active_inventory_package_with_incoming_transfer_package_records = active_inventory_packages_with_incoming_transfer_packages_dataframe.to_dict('records')

unique_sales_transaction_package_ids = set(sales_transactions_with_incoming_transfer_packages_dataframe.package_id.unique())
print(f'# unique sales transaction package ID: {len(unique_sales_transaction_package_ids)}')

match_count = 0
for active_inventory_package in active_inventory_package_with_incoming_transfer_package_records:
    package_id = active_inventory_package['package_id']
    package_type = active_inventory_package['type_l']
    packaged_date = active_inventory_package['packaged_date']
    shipped_quantity = active_inventory_package['shipped_quantity']
    current_quantity = active_inventory_package['quantity']
    if package_id in unique_sales_transaction_package_ids:
        match_count += 1
#         print(package_id, package_type, shipped_quantity, current_quantity)
    else:
        unit_of_measurement = active_inventory_package['unit_of_measure']
#         print(package_id, package_type, packaged_date, unit_of_measurement, quantity)

active_inventory_package_with_incoming_transfer_package_records_count = len(active_inventory_package_with_incoming_transfer_package_records)
print(f'# active inventory package with matching sales transaction(s): {match_count} ({match_count / active_inventory_package_with_incoming_transfer_package_records_count * 100}%)')
print(f'# active inventory package withOUT matching sales transaction(s): {active_inventory_package_with_incoming_transfer_package_records_count - match_count} ({(active_inventory_package_with_incoming_transfer_package_records_count - match_count) / active_inventory_package_with_incoming_transfer_package_records_count * 100}%)')

# unique sales transaction package ID: 736
# active inventory package with matching sales transaction(s): 683 (26.269230769230766%)
# active inventory package withOUT matching sales transaction(s): 1917 (73.73076923076923%)


In [63]:
from datetime import datetime

def compute_sell_through_for_package_id_records(package_id, package_id_records):
    created_date = None
    shipped_quantity = None
    product_category_name = None
    product_name = None
    
    # As of what date is X% of package is sold?
    percent_25_date = None
    percent_50_date = None
    percent_75_date = None
    percent_90_date = None
    percent_100_date = None
    
    # How many days until X% of package is sold?
    percent_25_days_count = None
    percent_50_days_count = None
    percent_75_days_count = None
    percent_90_days_count = None
    percent_100_days_count = None
    
    # As of date when X% of package is sold, on average how many units are sold per day?
    percent_25_days_count_norm = None
    percent_50_days_count_norm = None
    percent_75_days_count_norm = None
    percent_90_days_count_norm = None
    percent_100_days_count_norm = None

    running_gmv = 0
    running_quantity_sold = 0
    
    for package_id_record in package_id_records:
        if created_date is None:
            created_date = package_id_record['created_date']
            shipped_quantity = package_id_record['shipped_quantity']
            product_category_name = package_id_record['tx_product_category_name']
            product_name = package_id_record['tx_product_name']

        running_gmv += package_id_record['tx_total_price']
        running_quantity_sold += package_id_record['tx_quantity_sold']

        if shipped_quantity is None:
            print('BAD!')
            continue

        sales_date = package_id_record['sales_date']
        sold_ratio = running_quantity_sold / shipped_quantity
        
        if sold_ratio > 1.0:
            continue

        if percent_25_date is None and sold_ratio >= 0.25:
            percent_25_date = sales_date
            percent_25_days_count = (datetime.strptime(percent_25_date, '%Y-%m-%d') - created_date).days
            percent_25_days_count_norm = (shipped_quantity * 0.25) / (percent_25_days_count or 1)

        if percent_50_date is None and sold_ratio >= 0.5:
            percent_50_date = sales_date
            percent_50_days_count = (datetime.strptime(percent_50_date, '%Y-%m-%d') - created_date).days
            percent_50_days_count_norm = (shipped_quantity * 0.5) / (percent_50_days_count or 1)

        if percent_75_date is None and sold_ratio >= 0.75:
            percent_75_date = sales_date
            percent_75_days_count = (datetime.strptime(percent_75_date, '%Y-%m-%d') - created_date).days
            percent_75_days_count_norm = (shipped_quantity * 0.75) / (percent_75_days_count or 1)

        if percent_90_date is None and sold_ratio >= 0.9:
            percent_90_date = sales_date
            percent_90_days_count = (datetime.strptime(percent_90_date, '%Y-%m-%d') - created_date).days
            percent_90_days_count_norm = (shipped_quantity * 0.9) / (percent_90_days_count or 1)

        if percent_100_date is None and sold_ratio >= 1.0:
            percent_100_date = sales_date
            percent_100_days_count = (datetime.strptime(percent_100_date, '%Y-%m-%d') - created_date).days
            percent_100_days_count_norm = (shipped_quantity * 1.0) / (percent_100_days_count or 1)

    return [
        int(package_id),
        product_category_name,
        product_name,
        created_date.strftime('%Y-%m-%d'),
        shipped_quantity,
        running_quantity_sold,
        running_gmv / running_quantity_sold, # Average sales price per unit
        percent_25_date,
        percent_25_days_count,
        percent_25_days_count_norm,
        percent_50_date,
        percent_50_days_count,
        percent_50_days_count_norm,
        percent_75_date,
        percent_75_days_count,
        percent_75_days_count_norm,
        percent_90_date,
        percent_90_days_count,
        percent_90_days_count_norm,
        percent_100_date,
        percent_100_days_count,
        percent_100_days_count_norm,
    ]

def compute_sell_through_for_package_id(sales_transactions_dataframe, package_id):
    filter_by_package_id = sales_transactions_dataframe['package_id'] == package_id
    package_id_sales_transactions_dataframe = sales_transactions_dataframe[filter_by_package_id].sort_values(by=['sales_date'])
    package_id_sales_transaction_records = package_id_sales_transactions_dataframe.to_dict('records')
    if not package_id_sales_transaction_records:
        return None
    else:
        return compute_sell_through_for_package_id_records(package_id, package_id_sales_transaction_records)

In [64]:
# Compute sell throughs of all packages.

unique_sales_transaction_package_ids = sales_transactions_with_incoming_transfer_packages_dataframe.package_id.unique()

sell_throughs = []
for package_id in unique_sales_transaction_package_ids:
    sell_through = compute_sell_through_for_package_id(sales_transactions_with_incoming_transfer_packages_dataframe, package_id)
    if sell_through:
        sell_throughs += [sell_through]

filtered_package_with_sell_throughs_dataframe = pandas.DataFrame(
    sell_throughs,
    columns=[
        'package_id',
        'product_category_name',
        'product_name',
        'created_date',
        'shipped_quantity',
        'sold_quantity',
        'average_sales_price',
        'percent_25_date',
        'percent_25_days_count',
        'percent_25_days_count_norm',
        'percent_50_date',
        'percent_50_days_count',
        'percent_50_days_count_norm',
        'percent_75_date',
        'percent_75_days_count',
        'percent_75_days_count_norm',
        'percent_90_date',
        'percent_90_days_count',
        'percent_90_days_count_norm',
        'percent_100_date',
        'percent_100_days_count',
        'percent_100_days_count_norm',
    ])

print(f'# packages with sell through information: {len(filtered_package_with_sell_throughs_dataframe.index)}')

# packages with sell through information: 736


In [65]:
unique_product_names = filtered_package_with_sell_throughs_dataframe.product_name.unique()

product_name_with_average_sell_throughs = []
for product_name in unique_product_names:
    filtered_by_product_name_package_with_sell_throughs_dataframe = filtered_package_with_sell_throughs_dataframe[
        filtered_package_with_sell_throughs_dataframe['product_name'] == product_name
    ].sort_values(by=['created_date'])
    
    row_count = len(filtered_by_product_name_package_with_sell_throughs_dataframe.index)
    average_sales_price_mean = filtered_by_product_name_package_with_sell_throughs_dataframe['average_sales_price'].mean(skipna=True)
    average_sales_price_std = filtered_by_product_name_package_with_sell_throughs_dataframe['average_sales_price'].std(skipna=True)
    percent_50_days_norm_mean = filtered_by_product_name_package_with_sell_throughs_dataframe['percent_50_days_count_norm'].mean(skipna=True)
    percent_50_days_norm_std = filtered_by_product_name_package_with_sell_throughs_dataframe['percent_50_days_count_norm'].std(skipna=True)
    
    if row_count >= 3:
        product_name_with_average_sell_throughs += [(
            filtered_by_product_name_package_with_sell_throughs_dataframe.to_dict('records')[0]['product_category_name'],
            product_name,
            row_count,
            average_sales_price_mean,
            average_sales_price_std,
            percent_50_days_norm_mean,
            percent_50_days_norm_std,
        )]
    
product_name_with_average_sell_throughs_dataframe = pandas.DataFrame(
    product_name_with_average_sell_throughs,
    columns=[
        'product_category_name',
        'product_name',
        'row_count',
        'average_sales_price_mean',
        'average_sales_price_std',
        'percent_50_days_norm_mean',
        'percent_50_days_norm_std',
    ])

print(f'# product names with sell through information: {len(product_name_with_average_sell_throughs_dataframe.index)}')

# product names with sell through information: 12


In [66]:
product_name_with_average_sell_throughs_dataframe.sort_values(by=['percent_50_days_norm_mean'], ascending=False)

,product_category_name,product_name,row_count,average_sales_price_mean,average_sales_price_std,percent_50_days_norm_mean,percent_50_days_norm_std
9,Edible (weight - each),Lost Farm Chews Watermelon 100mg Gelato,3,22.782889,2.007021,0.588235,NaN
4,Vape Cartridge (volume - each),STIIIZY - Blue Dream Pod - 1.0g,5,43.761818,2.616733,0.400000,NaN
1,Vape Cartridge (volume - each),STIIIZY - Birthday Cake Pod - 1.0g,3,42.578824,5.739271,0.391414,0.232131
0,Vape Cartridge (volume - each),Dime Industries Bubblegum Kush 1000mg Tank,4,53.948333,2.793805,0.327273,0.179991
5,Vape Cartridge (volume - each),Bubblegum Kush 600mg Disposable,3,35.707037,4.139146,0.308333,0.153206
2,Flower (packaged eighth - each),Exotic Genetix - Gouda 8th Case(32-8th),3,57.266364,4.652256,NaN,NaN
3,Edible (weight - each),Kushy Punch Hybrid Gummy 100mg,3,12.544320,2.669505,NaN,NaN
6,Vape Cartridge (volume - each),Dime Industries Forbidden Fruit 600mg Disposable,3,36.573333,2.388751,NaN,NaN
7,Flower (packaged eighth - each),Exotic Genetix - Tina 8th Case(32-8th),3,57.522222,3.912137,NaN,NaN
8,Vape Cartridge (volume - each),Dime Industries Apple Gelato 1000mg Tank,3,51.416667,2.711242,NaN,NaN


In [67]:
grouped_active_inventory_package_with_incoming_transfer_packages_dataframe = active_inventory_packages_with_incoming_transfer_packages_dataframe.groupby(['product_category_name_l', 'product_name_l']).sum().reset_index()
print(f'# active inventory products: {len(grouped_active_inventory_package_with_incoming_transfer_packages_dataframe.index)}')
grouped_active_inventory_package_with_incoming_transfer_packages_dataframe_records = grouped_active_inventory_package_with_incoming_transfer_packages_dataframe.to_dict('records')

# active inventory products: 1892


In [68]:
grouped_active_inventory_package_with_incoming_transfer_packages_dataframe.columns

Index(['product_category_name_l', 'product_name_l', 'package_id', 'quantity',
       'manifest_number', 'shipper_wholesale_price', 'shipped_quantity',
       'is_testing_sample', 'is_trade_sample'],
      dtype='object')

In [69]:
product_name_with_runout_days = []

match_count = 0
for active_inventory_package in grouped_active_inventory_package_with_incoming_transfer_packages_dataframe_records:
    package_product_category_name = active_inventory_package['product_category_name_l']
    package_product_name = active_inventory_package['product_name_l']
    shipped_quantity = active_inventory_package['shipped_quantity']
    current_quantity = active_inventory_package['quantity']

    average_sell_throughs_dataframe = product_name_with_average_sell_throughs_dataframe[
        product_name_with_average_sell_throughs_dataframe['product_name'] == package_product_name
    ]
    if len(average_sell_throughs_dataframe.index) > 0:
        match_count += 1
        average_sell_through_record = average_sell_throughs_dataframe.to_dict('records')[0]
#             print(average_sell_through_record)
        historical_package_count = average_sell_through_record['row_count']
        average_sales_price_mean = average_sell_through_record['average_sales_price_mean']
        percent_50_days_norm_mean = average_sell_through_record['percent_50_days_norm_mean']
        runout_days = current_quantity / percent_50_days_norm_mean
        product_name_with_runout_days += [(
            package_product_category_name,
            package_product_name,
            historical_package_count,
            shipped_quantity,
            current_quantity,
            runout_days
        )]
#             print(package_product_category_name, package_product_name, shipped_quantity, current_quantity, runout_days)
#         else:
#             print(package_product_name)

#         print(package_id, package_type, shipped_quantity, current_quantity)
#     else:
#         unit_of_measurement = active_inventory_package['unit_of_measure']
#         print(package_id, package_type, packaged_date, unit_of_measurement, quantity)

active_inventory_package_with_incoming_transfer_package_records_count = len(active_inventory_package_with_incoming_transfer_package_records)
print(f'# package with matching sell through information: {match_count} ({match_count / active_inventory_package_with_incoming_transfer_package_records_count * 100}%)')
print(f'# package withOUT matching sell through information: {active_inventory_package_with_incoming_transfer_package_records_count - match_count} ({(active_inventory_package_with_incoming_transfer_package_records_count - match_count) / active_inventory_package_with_incoming_transfer_package_records_count * 100}%)')

product_name_with_runout_days_dataframe = pandas.DataFrame(
    product_name_with_runout_days,
    columns=[
        'package_product_category_name',
        'package_product_name',
        'historical_package_count',
        'shipped_quantity',
        'current_quantity',
        'runout_days'
    ])
product_name_with_runout_days_dataframe = product_name_with_runout_days_dataframe.sort_values(by=['runout_days'], ascending=True)

# package with matching sell through information: 13 (0.5%)
# package withOUT matching sell through information: 2587 (99.5%)


In [70]:
product_name_with_runout_days_dataframe

,package_product_category_name,package_product_name,historical_package_count,shipped_quantity,current_quantity,runout_days
11,Vape Cartridge (weight - each),STIIIZY - Birthday Cake Pod - 1.0g,3,20.0,3.0,7.664516
2,Edible (weight - each),Lost Farm Chews Watermelon 100mg Gelato,3,140.0,10.0,17.000000
10,Vape Cartridge (volume - each),STIIIZY - Blue Dream Pod - 1.0g,5,140.0,8.0,20.000000
12,Vape Cartridge (weight - each),STIIIZY - Blue Dream Pod - 1.0g,5,20.0,9.0,22.500000
6,Vape Cartridge (volume - each),Bubblegum Kush 600mg Disposable,3,70.0,8.0,25.945946
8,Vape Cartridge (volume - each),Dime Industries Bubblegum Kush 1000mg Tank,4,110.0,10.0,30.555556
0,Edible (weight - each),Kushy Punch Hybrid Gummy 100mg,3,161.0,139.0,NaN
1,Edible (weight - each),Kushy Punch Sugar Free Sativa Gummy 100mg,3,144.0,71.0,NaN
3,Extract (weight - each),Stone Road Concentrate Animal Cookies Sugar,3,40.0,2.0,NaN
4,Flower (packaged eighth - each),Exotic Genetix - Gouda 8th Case(32-8th),3,96.0,21.0,NaN


In [46]:
import time

xlsx_file_name = f'~/Downloads/product_name_with_runout_days_data_{round(time.time())}.xlsx'
product_name_with_runout_days_dataframe.to_excel(xlsx_file_name, index=False)